In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
date_envio = '2024-01-31'
monto_me_mail = 15241475.46
comision_bnte_mail = 3497.50
iva_bnte_mail = 559.60
monto_fondear_mail = 15245532.56
date_envio = pd.to_datetime(date_envio)

# READING FILES

## SANTANDER

In [3]:
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                  ,dtype={'BIN':object,'Clave de la institución':object,'Nombre de la institución':object})

In [4]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

In [5]:
query_1 = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\SANT-2 20*.csv', recursive=False)
query_1

['G:\\.shortcut-targets-by-id\\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\\3.-Settlement\\5.-HISTORICO FILES PAGOS\\01. TRADICIONAL\\2024\\01. JANUARY\\SANT-2 2024-01-31.csv']

In [6]:
query = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                   .assign(file_name=os.path.basename(f)) for f in query_1])
query.columns = query.columns.str.strip().str.lower() 
query['CLABE'] = query['bank_account_number'].str[1:]
query['BIN'] = query['bank_account_number'].str[1:4].replace(' ', '',regex=True)
query['date_disbursement'] = date_envio

In [7]:
query_final = query.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
query_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,...,bank_account_number,batch_created_date,disbursement_method,swap,file_name,CLABE,BIN,date_disbursement,Clave de la institución,Nombre de la institución
0,NaN,3df855dd-ffb0-4070-9fd3-353768283412,NaN,NaN,NaN,ADBQCVM4H,2024-01-28,"7,053.60",PAID,"Banco Santander, S.A.",...,'014180567990845339,2024-01-31T08:23:14Z,SANTANDER,"Juan Manuel Meliton Medina,,7053.6,01418056799...",SANT-2 2024-01-31.csv,014180567990845339,014,2024-01-31,40014,SANTANDER
1,NaN,ee69a0d5-effc-459c-9bad-f103fe3cf399,NaN,NaN,NaN,AKR4JT5TN,2024-01-31,615.22,PAID,BBVA Bancomer,...,'012694015433491060,2024-01-31T08:23:14Z,SANTANDER,"Farid Fregoso Velazque,,615.22,012694015433491...",SANT-2 2024-01-31.csv,012694015433491060,012,2024-01-31,40012,BBVA BANCOMER
2,NaN,ee6a9ee2-4599-4120-bdd5-cef65712004d,NaN,NaN,NaN,AFNDCT2LT,2024-01-31,798.34,PAID,BBVA Bancomer,...,'012180028688103031,2024-01-31T08:23:14Z,SANTANDER,"SALVADOR PANTOJA PAZ,,798.34,01218002868810303...",SANT-2 2024-01-31.csv,012180028688103031,012,2024-01-31,40012,BBVA BANCOMER
3,NaN,ee6b04f2-4baf-437b-803c-19b95888f7fb,NaN,NaN,NaN,AQHTPZ9R5,2024-01-31,258.60,PAID,BanCoppel,...,'137691101057707029,2024-01-31T08:23:14Z,SANTANDER,"Patricia Maria Lopez Alejandro,,258.6,13769110...",SANT-2 2024-01-31.csv,137691101057707029,137,2024-01-31,40137,BANCOPPEL
4,NaN,ee6b1ec9-5377-4f48-8b2e-d44ee8ade697,NaN,NaN,NaN,AM72587Q9,2024-01-31,11.50,PAID,Banco Nacional de México,...,'002420904824785722,2024-01-31T08:23:14Z,SANTANDER,"Diego Armando Martinez Alcantara,,11.5,0024209...",SANT-2 2024-01-31.csv,002420904824785722,002,2024-01-31,40002,BANAMEX


In [8]:
query_final.shape

(6995, 22)

# PAGOS DUPLICADOS 

In [9]:
duplicates_query = query_final[query_final.duplicated(['report_id'],keep=False)]
duplicates_query.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,...,bank_account_number,batch_created_date,disbursement_method,swap,file_name,CLABE,BIN,date_disbursement,Clave de la institución,Nombre de la institución


In [10]:
duplicates_query.shape

(0, 22)

# VALIDACION MONTO A DISPERSAR

In [11]:
valid_0 = query_final
valid_0.loc[valid_0['Nombre de la institución'].str.contains('SANTANDER'), 'comision_dsp'] = 0.50
valid_0.loc[~valid_0['Nombre de la institución'].str.contains('SANTANDER'), 'comision_dsp'] = 0.50
valid_0['iva_dsp'] = valid_0['comision_dsp']*0.1600

In [12]:
valid_1 = valid_0.groupby(['date_disbursement']).agg({'amount':sum,'report_id':'count','comision_dsp':sum
                                                      ,'iva_dsp':sum}).reset_index()
valid_1.rename(columns={'report_id':'payments_number'},inplace=True)
valid_1['comision + iva'] = valid_1['comision_dsp'] + valid_1['iva_dsp']
valid_1['total_fondeo'] = valid_1['amount'] + valid_1['comision + iva']
valid_1['monto_mail'] = monto_me_mail
valid_1['comision_mail'] = comision_bnte_mail
valid_1['iva_mail'] = iva_bnte_mail
valid_1['monto_fondeo_mail'] = monto_fondear_mail
valid_1['check_monto'] = valid_1['amount'] - valid_1['monto_mail']
valid_1['check_comision'] = valid_1['comision_dsp'] - valid_1['comision_mail']
valid_1['check_iva'] = valid_1['iva_dsp'] - valid_1['iva_mail']
valid_1['check_fondeo'] = valid_1['total_fondeo'] - valid_1['monto_fondeo_mail']
valid_1['dsp'] = 'SANT'
valid_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_15424\3280025601.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  valid_1 = valid_0.groupby(['date_disbursement']).agg({'amount':sum,'report_id':'count','comision_dsp':sum


,date_disbursement,amount,payments_number,comision_dsp,iva_dsp,comision + iva,total_fondeo,monto_mail,comision_mail,iva_mail,monto_fondeo_mail,check_monto,check_comision,check_iva,check_fondeo,dsp
0,2024-01-31,"15,241,475.46",6995,"3,497.50",559.60,"4,057.10","15,245,532.56","15,241,475.46","3,497.50",559.60,"15,245,532.56",-0.00,0.00,0.00,-0.00,SANT


In [13]:
valid_2 = valid_1[['date_disbursement','dsp','payments_number','amount','comision_dsp','iva_dsp','comision + iva','total_fondeo']]
valid_2.head()

,date_disbursement,dsp,payments_number,amount,comision_dsp,iva_dsp,comision + iva,total_fondeo
0,2024-01-31,SANT,6995,"15,241,475.46","3,497.50",559.60,"4,057.10","15,245,532.56"


In [14]:
valid_montos_final = valid_2.stack().reset_index(1)
valid_montos_final.rename(columns={'level_1':'concept',0:'data'},inplace=True)
valid_montos_final

,concept,data
0,date_disbursement,2024-01-31 00:00:00
0,dsp,SANT
0,payments_number,6995
0,amount,"15,241,475.46"
0,comision_dsp,"3,497.50"
0,iva_dsp,559.60
0,comision + iva,"4,057.10"
0,total_fondeo,"15,245,532.56"


# EXCEL

In [15]:
writer = pd.ExcelWriter(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\VALIDATION\VALIDATION-SANT-2 2024-01-31.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
query.to_excel(writer, sheet_name='DATA QUERY',index=False)
valid_montos_final.to_excel(writer, sheet_name='MONTOS',index=False)
valid_1.to_excel(writer, sheet_name='DIFF MONTOS',index=False)
duplicates_query.to_excel(writer, sheet_name='PAGOS DUPLICADOS',index=False)
writer.close()

In [16]:
valid_1.to_csv(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\MONTOS EMAIL\DIFF MONTOS-SANT-2 2024-01-31.csv',index=False)

In [17]:
valid_2.to_csv(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\VALIDATION\total_disbursement-SANT-2 2024-01-31.csv',index=False)

In [18]:
duplicates_query.to_csv(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2024\01. JANUARY\VALIDATION\Duplicados-SANT-2 2024-01-31.csv',index=False)